# RBF与DemBones集成演示

这个Jupyter Notebook展示了如何结合DemBones和SciPy的RBF功能，实现类似于Chad Vernon的RBF节点功能。我们将使用DemBones计算骨骼权重和变换，然后使用RBF插值器驱动辅助关节。

## 安装依赖

首先，我们需要安装必要的依赖项：

In [ ]:
!pip install py-dem-bones numpy scipy matplotlib

## 导入必要的库

In [ ]:
import numpy as np
from scipy.interpolate import RBFInterpolator
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import py_dem_bones as pdb

## 创建测试数据

我们将创建一个简单的立方体网格和骨骼系统来演示RBF功能：

In [ ]:
def create_cube_vertices(size=2.0):
    """创建一个简单的立方体网格顶点"""
    vertices = np.array([
        [-1, -1, -1],  # 前下左
        [1, -1, -1],   # 前下右
        [1, 1, -1],    # 前上右
        [-1, 1, -1],   # 前上左
        [-1, -1, 1],   # 后下左
        [1, -1, 1],    # 后下右
        [1, 1, 1],     # 后上右
        [-1, 1, 1],    # 后上左
    ]) * size/2
    return vertices

def create_joints():
    """创建测试用的骨骼位置"""
    joints = np.array([
        [-1, 0, 0],  # 根关节
        [1, 0, 0],   # 末端关节
    ])
    return joints

# 创建测试数据
vertices = create_cube_vertices()
joints = create_joints()

# 可视化网格和骨骼
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# 绘制顶点
ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c='b', marker='o')

# 绘制骨骼
ax.plot(joints[:, 0], joints[:, 1], joints[:, 2], 'r-', linewidth=2)
ax.scatter(joints[:, 0], joints[:, 1], joints[:, 2], c='r', marker='s')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('测试网格和骨骼系统')
plt.show()

## 设置DemBones

现在我们将设置DemBones系统并计算蒙皮权重：

In [ ]:
# 创建DemBones实例
dem_bones = pdb.DemBones()

# 设置参数
dem_bones.nV = len(vertices)  # 顶点数量
dem_bones.nB = len(joints)    # 骨骼数量
dem_bones.nnz = 4             # 每个顶点最大影响数
dem_bones.weightsSmooth = 0.0001 * 3  # 权重平滑因子

# 设置顶点位置
dem_bones.u = vertices.astype(np.float64).flatten()

# 计算权重
dem_bones.compute()

# 获取计算的权重
weights = dem_bones.get_weights()

# 可视化权重
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 第一个骨骼的权重
ax1.scatter(vertices[:, 0], vertices[:, 1], c=weights[0], cmap='viridis')
ax1.set_title('骨骼1的权重')

# 第二个骨骼的权重
ax2.scatter(vertices[:, 0], vertices[:, 1], c=weights[1], cmap='viridis')
ax2.set_title('骨骼2的权重')

plt.colorbar(ax1.collections[0], ax=ax1)
plt.colorbar(ax2.collections[0], ax=ax2)
plt.show()

## 设置RBF插值

现在我们将设置RBF插值器来驱动辅助关节：

In [ ]:
def create_rbf_interpolator(key_poses, key_values, rbf_function='thin_plate_spline'):
    """创建RBF插值器
    
    参数：
        key_poses: 关键姿势的输入值 (n_samples, n_features)
        key_values: 每个关键姿势对应的输出值 (n_samples, m)
        rbf_function: RBF函数类型
    """
    return RBFInterpolator(
        key_poses, 
        key_values,
        kernel=rbf_function
    )

# 定义关键姿势
key_poses = np.array([
    [0.0, 0.0],  # 默认姿势
    [1.0, 0.0],  # X方向极值
    [0.0, 1.0],  # Y方向极值
])

# 定义对应的辅助关节位置
key_values = np.array([
    # 默认姿势的辅助关节位置
    [[0.5, 0.5, 0.0], [0.5, 0.5, 1.0]],
    # X方向极值的辅助关节位置
    [[0.7, 0.5, 0.0], [0.7, 0.5, 1.2]],
    # Y方向极值的辅助关节位置
    [[0.5, 0.7, 0.0], [0.5, 0.7, 1.2]],
])

# 创建RBF插值器
rbf = create_rbf_interpolator(
    key_poses,
    key_values.reshape(3, -1),
    rbf_function='thin_plate_spline'
)

# 创建测试网格来可视化RBF插值效果
x = np.linspace(-0.5, 1.5, 20)
y = np.linspace(-0.5, 1.5, 20)
X, Y = np.meshgrid(x, y)
test_poses = np.column_stack((X.ravel(), Y.ravel()))

# 计算插值结果
interpolated = rbf(test_poses).reshape(-1, 2, 3)

# 可视化插值结果
fig = plt.figure(figsize=(15, 5))

# 第一个辅助关节的位置
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(interpolated[:, 0, 0], interpolated[:, 0, 1], interpolated[:, 0, 2], c=test_poses[:, 0])
ax1.set_title('辅助关节1的插值位置')

# 第二个辅助关节的位置
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(interpolated[:, 1, 0], interpolated[:, 1, 1], interpolated[:, 1, 2], c=test_poses[:, 1])
ax2.set_title('辅助关节2的插值位置')

# 组合视图
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(interpolated[:, 0, 0], interpolated[:, 0, 1], interpolated[:, 0, 2], c='r', label='Joint 1')
ax3.scatter(interpolated[:, 1, 0], interpolated[:, 1, 1], interpolated[:, 1, 2], c='b', label='Joint 2')
ax3.set_title('组合视图')
ax3.legend()

plt.tight_layout()
plt.show()

## 尝试不同的RBF核函数

SciPy的RBFInterpolator支持多种核函数，让我们比较它们的效果：

In [ ]:
kernels = [
    'thin_plate_spline',
    'multiquadric',
    'inverse_multiquadric',
    'gaussian',
    'linear',
    'cubic',
    'quintic'
]

fig = plt.figure(figsize=(20, 10))

for i, kernel in enumerate(kernels):
    rbf = create_rbf_interpolator(key_poses, key_values.reshape(3, -1), kernel)
    interpolated = rbf(test_poses).reshape(-1, 2, 3)
    
    ax = fig.add_subplot(2, 4, i+1, projection='3d')
    ax.scatter(interpolated[:, 0, 0], interpolated[:, 0, 1], interpolated[:, 0, 2], c='r', alpha=0.5)
    ax.scatter(interpolated[:, 1, 0], interpolated[:, 1, 1], interpolated[:, 1, 2], c='b', alpha=0.5)
    ax.set_title(f'核函数: {kernel}')

plt.tight_layout()
plt.show()

## 总结

在这个演示中，我们展示了：

1. 如何使用DemBones计算骨骼权重
2. 如何使用RBF插值器创建平滑的辅助关节动画
3. 不同RBF核函数的效果比较

这些技术可以应用于：

- 面部表情动画
- 次级动画控制
- 姿势空间变形
- 其他需要平滑插值的动画场景

选择合适的RBF核函数取决于你的具体需求：

- `thin_plate_spline`: 适合大多数情况，提供平滑的插值
- `multiquadric`: 类似于thin_plate_spline，但有不同的衰减特性
- `gaussian`: 局部影响更强，适合需要局部控制的情况
- `linear`: 最简单的线性插值，计算快但可能不够平滑